In [25]:
#ACHTUNG: save summary fehlt noch!! 

In [15]:
#!uv remove openai

In [16]:
#!uv add openai-agents

In [1]:
import agents
agents.__version__

'0.4.2'

In [2]:
from agents import Agent, function_tool, Runner


In [3]:
#url = "https://en.wikipedia.org/wiki/Capybara"

### SearchAgent

In [29]:
#dir(search_tools.index)
search_tools.index.docs[0]

{'start': 0,
 'content': 'Title: Lesser capybara\n\nURL Source: https://en.wikipedia.org/wiki/Lesser_capybara\n\nPublished Time: 2009-06-12T03:46:13Z\n\nMarkdown Content:\nLesser capybara - Wikipedia\n\n===============\n[Jump to content](https://en.wikipedia.org/wiki/Lesser_capybara#bodyContent)\n\n- [x] Main menu \n\nMain menu\n\nmove to sidebar hide\n\n Navigation \n\n*   [Main page](https://en.wikipedia.org/wiki/Main_Page "Visit the main page [alt-z]")\n*   [Contents](https://en.wikipedia.org/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")\n*   [Current events](https://en.wikipedia.org/wiki/Portal:Current_events "Articles related to current events")\n*   [Random article](https://en.wikipedia.org/wiki/Special:Random "Visit a randomly selected article [alt-x]")\n*   [About Wikipedia](https://en.wikipedia.org/wiki/Wikipedia:About "Learn about Wikipedia and how it works")\n*   [Contact us](https://en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")\n\n Contrib

In [6]:
from minsearch import AppendableIndex
import docs
from typing import Any, Dict, List, Optional

class SearchTools:

    def __init__(self) -> None:
        self.index = AppendableIndex(text_fields=['content'])

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the index for documents matching a query string.

        Args:
            query (str): The search query.

        Returns:
            List[Dict[str, Any]]: A list of search result dictionaries.
        """
        return self.index.search(query, num_results=5)

    def index_content(self, url: str, content: Optional[str] = None) -> str:
        """
        Index text content from a given source into the search index.

        Args:
            video_id (str): video_id or link to the web page.
            content (str): transcript for the provided video_id or None if the transcript needs to be downloaded

        Returns:
            str: "SUCCESS" upon successful indexing.
        """
        if content is None:
            content = summary_tools.fetch_url(url)
        chunks = docs.sliding_window(content, size=3000, step=1500)

        for chunk in chunks:
            chunk["url"] = url
            self.index.append(chunk)

        return "SUCCESS"

In [7]:
search_tools= SearchTools()

In [8]:
#search_tools.index_content('https://en.wikipedia.org/wiki/Capybara')

In [9]:
#search_tools.search('herbivores')

In [10]:
from toyaikit.tools import wrap_instance_methods

In [11]:
search_method_tools = wrap_instance_methods(function_tool, search_tools)

In [12]:
search_instructions = """
Your task is to search through indexed documents.

Before performing a search:

1. Check if the document has been indexed.
2. If not, call `index_content` to index it.

Important rules:

- Do NOT call `search` using a webpage URL, or any direct link as the query.
- After successfully indexing a source, do NOT perform a search automatically.
- Instead, tell the user something like:
  "A page about this topic has been indexed. What would you like to learn about it?"

Only perform a search if the user provides a natural-language question or topic of interest
(e.g., "What does the page say about AI safety?"), not a URL or ID.

Do not attempt to summarize content.
"""

search_agent = Agent(
    name='search_agent',
    instructions=search_instructions,
    handoff_description="Whenever the user needs to search for things about the content of a url",
    tools=search_method_tools,
    model='gpt-4o-mini'
)

In [13]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=search_agent
)

In [14]:
#await runner.run();

### Summary Agent

In [26]:
import requests
from requests.exceptions import RequestException, Timeout, HTTPError
class SummaryTools:

    def __init__(self, search_tools):
        self.search_tools = search_tools

    def fetch_url(self,url: str) -> str:
        """
        Fetches the content of a given URL by appending it to the base URL. appends the given URL EXACTLY to the base_url and performs an HTTP GET request.
    
        Args:
            url (str): The absolute URL path to fetch.
            
    
        Returns:
            str: The decoded text content of the response.
    
        Raises:
            ValueError: If the URL or response content is invalid or cannot be decoded.
            HTTPError: If the HTTP request returns an unsuccessful status code.
            Timeout: If the request times out.
            RequestException: For general network-related errors.
        """
        base_url = 'https://r.jina.ai/'
        jina_reader_url = base_url + url
        try:
            response = requests.get(jina_reader_url, timeout=30)
            response.raise_for_status()  # Raises HTTPError for bad status codes
            try:
                content = response.content.decode("utf-8")
                self.search_tools.index_content(url, content)
                return content
            except UnicodeDecodeError as e:
                raise ValueError(f"Failed to decode response content from {jina_reader_url}") from e
        except Timeout as e:
            raise Timeout(f"Request to {jina_reader_url} timed out.") from e
        except HTTPError as e:
            raise HTTPError(f"HTTP error occurred while fetching {jina_reader_url}: {e}") from e
        except RequestException as e:
            raise RequestException(f"Error occurred while requesting {jina_reader_url}: {e}") from e


    #def save_summary():
        

    #def fetch_youtube_transcript(self, video_id: str) -> str:
    #    """
    #    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.
   # 
   #     Args:
   #         video_id (str): The unique YouTube video ID.
   # 
   #     Returns:
   #         str: The subtitles generated from the video's transcript.
   #     """
   #     content = youtube.fetch_transcript_cached(video_id)
   #     self.search_tools.index_video(video_id, content)
   #     return content

In [16]:
summary_tools = SummaryTools(search_tools)

In [17]:
summary_instructions = """
You're a helpful assistant that summarizes webpages
"""

tools = [
    function_tool(summary_tools.fetch_url)
]

summary_agent = Agent(
    name='summary agent',
    tools=tools,
    handoff_description="Whenever the user needs a summary of the webpage",
    instructions=summary_instructions,
    model='gpt-4o-mini'
)

from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=summary_agent
)

#await runner.run();

### Orchestrator

In [18]:
from agents import handoff

In [19]:
triage_instructions = """
You are the orchestrator between two specialized agents:

1. summarizing_agent — summarizes web pages 
2. search_agent — searches within previously indexed documents to answer detailed or follow-up questions.

Routing rules:

- If the user sends a url without any question → hand off to summary_agent.
- If the user asks for a summary, overview, or "what is this video about" → summary_agent.
- If the user asks a follow-up question after a summary (e.g. "how did they do X?", "what does she say about Y?"),
  or refers to something mentioned in a previously summarized or indexed resource → hand off to search_agent.
- If the user asks a direct content question about a topic inside a resource → search_agent.
- Prefer delegating the answer to agents when possible. 

Examples:
User: "https://en.wikipedia.org/wiki/Capybara"
→ summary_agent

User: "What does the page say about their food?"
→ search_agent

User: "How exactly do they sleep?" (after a summary)
→ search_agent
""".strip()

triage_agent = Agent(
    name='triage_agent',
    instructions=triage_instructions,
    handoffs=[
        handoff(summary_agent, on_handoff=lambda ctx: print('handoff to summary agent')),
        handoff(search_agent, on_handoff=lambda ctx: print('handoff to search agent')),
    ],
    model='gpt-4o-mini'
)

In [30]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=triage_agent
)

await runner.run();

You: https://en.wikipedia.org/wiki/Neochoerus


handoff to summary agent
handoff: transfer_to_summary_agent
handoff: summary agent -> triage_agent successful


You: https://en.wikipedia.org/wiki/Caviodon


handoff to summary agent
handoff: transfer_to_summary_agent
handoff: summary agent -> triage_agent successful


You: https://en.wikipedia.org/wiki/Neochoerus_aesopi


...

handoff to summary agent
handoff: transfer_to_summary_agent
handoff: summary agent -> triage_agent successful


You: What are threats to capybara populations?


handoff to search agent
handoff: transfer_to_search_agent
handoff: search_agent -> triage_agent successful


You: stop


Chat ended.
